# Welcome to Session 7 - Working with Classes

A lot of the text and example code below is taken directly or modified from the Python Documentation (see https://docs.python.org/3/tutorial/classes.html ) 

Classes provide a means of bundling data and functionality together. Creating a new class creates a new type of object, allowing new instances of that type to be made. Each class instance can have attributes attached to it for maintaining its state. Class instances can also have methods (defined by its class) for modifying its state.

Compared with other programming languages, Python’s class mechanism adds classes with a minimum of new syntax and semantics. It is a mixture of the class mechanisms found in C++ and Modula-3. Python classes provide all the standard features of Object Oriented Programming: the class inheritance mechanism allows multiple base classes, a derived class can override any methods of its base class or classes, and a method can call the method of a base class with the same name. Objects can contain arbitrary amounts and kinds of data. As is true for modules, classes partake of the dynamic nature of Python: they are created at runtime, and can be modified further after creation.

### Making an object

We have been working with Python classes for a long time, we just might not have known it!

In [1]:
TestInt = 5
print(type(TestInt))  # TestInt is a class integer

<class 'int'>


In [6]:
print(f'The value of 2 + 3 is {2 + 3}')  # how does the language know how to add two integers together?
print(int.__add__(2,3))

The value of 2 + 3 is 5
5


In [5]:
import inspect
print(inspect.getmembers(TestInt))

[('__abs__', <method-wrapper '__abs__' of int object at 0x7f9d10658170>), ('__add__', <method-wrapper '__add__' of int object at 0x7f9d10658170>), ('__and__', <method-wrapper '__and__' of int object at 0x7f9d10658170>), ('__bool__', <method-wrapper '__bool__' of int object at 0x7f9d10658170>), ('__ceil__', <built-in method __ceil__ of int object at 0x7f9d10658170>), ('__class__', <class 'int'>), ('__delattr__', <method-wrapper '__delattr__' of int object at 0x7f9d10658170>), ('__dir__', <built-in method __dir__ of int object at 0x7f9d10658170>), ('__divmod__', <method-wrapper '__divmod__' of int object at 0x7f9d10658170>), ('__doc__', "int([x]) -> integer\nint(x, base=10) -> integer\n\nConvert a number or string to an integer, or return 0 if no arguments\nare given.  If x is a number, return x.__int__().  For floating point\nnumbers, this truncates towards zero.\n\nIf x is not a number or if base is given, then x must be a string,\nbytes, or bytearray instance representing an integer l

### Creating a blank class instance and assigning variables

Making a blank Dolphin record and then adding fields

In [8]:
import datetime

class Dolphin:
    pass

Dolphin_327 = Dolphin()  # Create an empty dolphin record

# Fill the fields of the record
Dolphin_327.name = 'CrookedFin'
Dolphin_327.stock = 'Ashley River'
Dolphin_327.born = datetime.datetime.strptime('10/25/2001', '%m/%d/%Y')

Print the data

In [9]:
# Output the data
print(f'Name: {Dolphin_327.name}, Stock: {Dolphin_327.stock}, Birth: {Dolphin_327.born}')

Name: CrookedFin, Stock: Ashley River, Birth: 2001-10-25 00:00:00


### Defining a simple class with variables and functions

In [10]:
class Fisherperson:
    """A simple example class"""
    NumberCaught = 12345     # i for a generic integer

    def Fisher(self):   # f for a generic function
        return 'All fish caught by College of Charleston'

Class instantiation

In [11]:
StudentA = Fisherperson()

Print a variable in the class

In [12]:
print(f'The number of fish caught: {StudentA.NumberCaught:,}')

The number of fish caught: 12,345


Run a function from the class object

In [13]:
StudentA.Fisher()

'All fish caught by College of Charleston'

### Class and Instance Variables

Making a class MarineMammal

In [14]:
class MarineMammal:

    Phylum = 'Chordata'         # class variable shared by all instances

    def __init__(self, species):
        self.species = species    # instance variable unique to each instance

You have 2 new specimens in the lab, how to store information about them?   Make an instance!

In [16]:
Specimen1 = MarineMammal('Tursiops truncatus')       # bottlenose dolphin
Specimen2 = MarineMammal('Kogia breviceps')          # pygmy sperm whale

What species of each instance?

In [17]:
print(Specimen1.species)
print(Specimen1.Phylum)
print('--------------')
print(Specimen2.species)
print(Specimen2.Phylum)

Tursiops truncatus
Chordata
--------------
Kogia breviceps
Chordata


### Class variables - list concerns

In [18]:
class MarineMammal:
    
    SamplesCollected = []
    Phylum = 'Chordata'         # class variable shared by all instances

    def __init__(self, species):
        self.species = species    # instance variable unique to each instance
        
    def add_samples(self, sample):
        self.SamplesCollected.append(sample)

        
Specimen1 = MarineMammal('Tursiops truncatus')       # bottlenose dolphin
Specimen2 = MarineMammal('Kogia breviceps')          # pygmy sperm whale
Specimen1.add_samples('heart')
Specimen2.add_samples('brain')

What would be the output for the following?

In [19]:
Specimen1.SamplesCollected

['heart', 'brain']

### Fixing the class list variable issue by using an instance variable

In [20]:
class MarineMammal:

    Phylum = 'Chordata'                              # class variable shared by all instances

    def __init__(self, species):
        self.species = species                       # instance variable unique to each instance
        self.SamplesCollected = []
            
    def add_samples(self, sample):
        self.SamplesCollected.append(sample)

        
Specimen1 = MarineMammal('Tursiops truncatus')       # bottlenose dolphin
Specimen2 = MarineMammal('Kogia breviceps')          # pygmy sperm whale
Specimen1.add_samples('heart')
Specimen2.add_samples('brain')

What is the output of the following?

In [21]:
Specimen1.SamplesCollected

['heart']

In [22]:
Specimen2.SamplesCollected

['brain']

### Class and instance variables - integers

The following sections are loosely based on some examples from Chapter 6 of "Learn Python Programming - Second Edition" written by Fabrizio Romano,  see:  https://www.packtpub.com/free-ebook/learn-python-programming-second-edition/9781788996662

In [24]:
class SeaBass:
    AgeAtCollection = 8
    
    def AgeMonths(self): 
        return self.AgeAtCollection * 12
    
    def PrintInfo(self): 
        return print(f'age at collection was {self.AgeAtCollection} years which is {self.AgeMonths()} months') 
    

Sample1 = SeaBass()
print(f'Sample 1: ')
Sample1.PrintInfo()
print()

# access the class variable

Sample2 = SeaBass()
Sample2.AgeAtCollection = 10       # update the class variable to an instance
print('Sample 2: ')
Sample2.PrintInfo()                # access the instance variable


Sample 1: 
age at collection was 8 years which is 96 months

Sample 2: 
age at collection was 10 years which is 120 months


What happens to the AgeAtCollection for sample 1?

In [25]:
# Check Age for Sample 1
print('Sample 1: ')
Sample1.PrintInfo()  # access the class variable

Sample 1: 
age at collection was 8 years which is 96 months


What happens to the AgeAtCollection for a new sample?

In [26]:
Sample3 = SeaBass()
print('Sample 3: ') 
Sample3.PrintInfo()  # access the class variable

Sample 3: 
age at collection was 8 years which is 96 months


### Use of class __init__ to assign values

In [27]:
class SeaBass:
    def __init__(self, SampleLetter, AgeAtCollection = 8, CollectionSite = 'Reef_A'):
        self.AgeAtCollection = AgeAtCollection
        self.SampleLetter = SampleLetter
        self.CollectionSite = CollectionSite
    
    def AgeMonths(self): 
        return self.AgeAtCollection * 12
    
    def PrintInfo(self): 
        return print(f"\nSamp1e {self.SampleLetter}'s age at collection was {self.AgeAtCollection} years which is {self.AgeMonths()} months") 
    

SampleA = SeaBass('A', CollectionSite = 'Savannah')

SampleB = SeaBass('B', 10, CollectionSite = 'Edisto')

What happens here?

In [28]:
SampleA.PrintInfo()


Samp1e A's age at collection was 8 years which is 96 months


What happens here?

In [29]:
SampleB.PrintInfo()


Samp1e B's age at collection was 10 years which is 120 months


What happens here?

In [30]:
# Change collection site
print(f'Sample{SampleA.SampleLetter} was misclassified as being collected in {SampleA.CollectionSite}') 
SampleA.CollectionSite = 'Charleston'
print(f'Sample{SampleA.SampleLetter} was really collected in {SampleA.CollectionSite}')  
print(SampleB.CollectionSite)

SampleA was misclassified as being collected in Savannah
SampleA was really collected in Charleston
Edisto


### Working with Getters and Setters - ways to control access to private variables and validate input

Define 3 different input classes to test each one

In [34]:
class Whale:
    def __init__(self, size):
        self.size = size  # anyone can modify this freely

class WhaleWithAccessors:
    def __init__(self, size):
        self._size = size

    def get_size(self):
        return self._size

    def set_size(self, size):
        if size in ["big", "medium", "small"]:
            self._size = size
        else:
            raise ValueError('size must be big, medium, or small')

class WhaleWithProperties:
    def __init__(self):
        self._size = None

    @property
    def size(self):
        return self._size

    @size.setter
    def size(self, inputsize):
        if inputsize in ["big", "medium", "small"]:
            self._size = inputsize
        else:
            raise ValueError('size must be big, medium, or small')

Test of the Whale class - What happens here?

In [35]:
class Whale:
    def __init__(self, size):
        self.size = size  # anyone can modify this freely

whale0 = Whale('Tiny')
print(f'whale0 is size: {whale0.size}')
whale0.size = 'ReallyTiny'
print(f'whale0 is now size: {whale0.size} - NOTE SIZE CAN BE MODIFIED DIRECTLY')

whale0 is size: Tiny
whale0 is now size: ReallyTiny - NOTE SIZE CAN BE MODIFIED DIRECTLY


Test of the WhaleWithAccessors class - What happens here?

In [36]:
class WhaleWithAccessors:
    def __init__(self, size):
        self._size = size

    def get_size(self):
        return self._size

    def set_size(self, size):
        if size in ["big", "medium", "small"]:
            self._size = size
        else:
            raise ValueError('size must be big, medium, or small')

whale1 = WhaleWithAccessors('Tiny')
print(f'whale1 is size: {whale1._size}')
print('\nTry to directly change size with size not in list...\n')
whale1._size = 'ReallyTiny'
print(f'whale1 is now size: {whale1.get_size()} - NOTE SIZE CAN STILL BE MODIFIED DIRECTLY')

print('\n\nTry to change size with set_size with size in list...\n')
whale1.set_size('big')
print(f'whale1 is now size: {whale1.get_size()} - WORKS!')

print('\n\nTry to change size with set_size with size not in list...\n')
whale1.set_size('SuperTiny')

whale1 is size: Tiny

Try to directly change size with size not in list...

whale1 is now size: ReallyTiny - NOTE SIZE CAN STILL BE MODIFIED DIRECTLY


Try to change size with set_size with size in list...

whale1 is now size: big - WORKS!


Try to change size with set_size with size not in list...



ValueError: size must be big, medium, or small

Test of the WhaleWithProperties class - What happens here?

In [38]:
class WhaleWithProperties:
    def __init__(self):
        self._size = None

    @property
    def size(self):
        return self._size

    @size.setter
    def size(self, inputsize):
        if inputsize in ["big", "medium", "small"]:
            self._size = inputsize
        else:
            raise ValueError('size must be big, medium, or small')

whale2 = WhaleWithProperties()

print('Set to size in list...\n')
whale2.size = 'big'
print(f'whale2 is now size: {whale2.size} - WORKS!')

print('\nNow try to directly change size with size not in list...\n')
whale2.size = 'ReallyTiny'

#whale2._size = 'ReallyTiny'   # NOTE THE _size
#print(f"whale2 is now size: {whale2.size} - WORKS, BUT YOU SHOULDN'T DO IT THIS WAY!")

Set to size in list...

whale2 is now size: big - WORKS!

Now try to directly change size with size not in list...

whale2 is now size: ReallyTiny - WORKS, BUT YOU SHOULDN'T DO IT THIS WAY!


### Assigning new class operands

In [39]:
class Trawl:
    def __init__(self, s = 50):
        self._TonnesOfFish = s

    def __bool__(self):
        return self._TonnesOfFish < 100


trawl1 = Trawl()

What happens here?

In [40]:
print(bool(0))
print(bool(200))
print(f'\n------------------\n')

print(f'Trawl was less than 100 tonnes:  {bool(trawl1)}')  # True

False
True

------------------

Trawl was less than 100 tonnes:  True


What happens here?

In [41]:
trawl2 = Trawl(200)
print(trawl2._TonnesOfFish)  
print(f'Trawl was less than 100 tonnes:  {bool(trawl2)}')  # False

200
Trawl was less than 100 tonnes:  False


### Inheritance - how to include one class within another

In [42]:
class Coral:
    def __init__(self, depth):
        self._depth = depth

    def PrintDepth(self):
        print(f'The coral was collected at {self._depth} feet')

class HardCoral(Coral):
    def __init__(self, depth, CoralType):
        super().__init__(depth)
        self._CoralType = CoralType

    def PrintCoralType(self):
        print(f'The coral type collected was {self._CoralType}')


coral1 = HardCoral(100, 'brain')

What happens here?

In [43]:
print(coral1._CoralType)
print(coral1._depth)

brain
100


What happens here?

In [44]:
coral1.PrintCoralType()
coral1.PrintDepth()

The coral type collected was brain
The coral was collected at 100 feet


### Use of class methods to make a class instance

@classmethod is a decorator function that operates on the class itself rather than an instance of the class.  Note it references cls instead of self.

In [45]:
class Location:

    def __init__(self, Lat, Long):
        self.Lat = Lat
        self.Long = Long

    @classmethod
    def from_list(cls, coords):  
        print('In classmethod: from_list')
        print(f'coords = {coords}')
        print(f'    coords[0] = {coords[0]}')
        print(f'    coords[1] = {coords[1]}')
        print('*coords = ', end = '')       # Note the '*' which is from *args to refer to multiple intput
        print(*coords)     
        return cls(coords[0], coords[1])    # could also use:   return cls(*coords)
        # return cls(*coords)

    @classmethod
    def from_class_location(cls, Location):  
        print('In classmethod: from_class_location')
        return cls(Location.Lat, Location.Long)


Calling a class method to make a class instance - example 1

In [53]:
location1 = Location.from_list([32.77, -79.93])
print(f'Location1 is {location1.Lat} N and {location1.Long} W\n')

In classmethod: from_list
coords = [32.77, -79.93]
    coords[0] = 32.77
    coords[1] = -79.93
*coords = 32.77 -79.93
Location1 is 32.77 N and -79.93 W



Calling a class method to make a class instance - example 2

In [55]:
location2 = Location.from_class_location(location1)
print(f'Location2 is {location2.Lat} N and {location2.Long} W\n')

#location1.Lat = 55.00
#print(f'Location1 is {location1.Lat} N and {location1.Long} W')
#print(f'Location2 is {location2.Lat} N and {location2.Long} W\n')

In classmethod: from_class_location
Location2 is 55.0 N and -79.93 W

Location1 is 55.0 N and -79.93 W
Location2 is 55.0 N and -79.93 W



### Use of static methods to make a namespace

@staticmethod is a decorator function that operates independantly of the class or instance.  Note it does not reference cls or self.

In [56]:
class WordUtilities:   #creates a WordUtilities namespace

    @staticmethod
    def print_first_word(sentence):
        words = sentence.split()
        return words[0]

    @staticmethod
    def print_second_word(sentence):
        words = sentence.split()
        return words[1]
    
    @staticmethod
    def print_longest_word(sentence):
        words = sentence.split()
        words = sorted(words, key = len)  # order of their lengths
        return words[-1]   # returns the last word in the list

Example using the WordUtilities namespace

In [57]:
sentence = 'Big fish live in the lake by the alligator with 2 teeth'
print(f'The sentence is : {sentence}\n')
print(f'The first word in the sentence is:  {WordUtilities.print_first_word(sentence)}')
print(f'The second word in the sentence is:  {WordUtilities.print_second_word(sentence)}')
print(f'The longest word in the sentence is:  {WordUtilities.print_longest_word(sentence)}')

The sentence is : Big fish live in the lake by the alligator with 2 teeth

The first word in the sentence is:  Big
The second word in the sentence is:  fish
The longest word in the sentence is:  alligator


### Use of dataclass

@dataclass is a decorator function that is formatted differently for storing data attributes

In [58]:
from dataclasses import dataclass
import datetime

@dataclass
class Fish:
    name: str
    length_inches: float = 0.0
    weight: float = 0.0
    collection: datetime.date = datetime.date(2019, 4, 13)

    def length_cm(self) -> float:
        return (self.length_inches * 2.54)


Class instance and example

In [59]:
fish0 = Fish('Tuna', 45.00, 175.50)
print(f'The type of fish created was: {fish0.name}')
print(f'The length of the fish created was: {fish0.length_inches} inches or {fish0.length_cm()} cm')
print(f'The weight of the fish created was: {fish0.weight} pounds')
print(f'The fish was collected on: {fish0.collection}')

The type of fish created was: Tuna
The length of the fish created was: 45.0 inches or 114.3 cm
The weight of the fish created was: 175.5 pounds
The fish was collected on: 2019-04-13


### Summative Assessment Quiz

The purpose of summative assessment quizzes is twofold:

1) The process of recall helps to transfer information from short term to longer term memory.
2) The quizzes help us evaluate the effectiveness of our training sessions.

Take [Summative Assessment Quiz 7](https://cofc.libwizard.com/f/intro-python-7) to test your knowledge about this session.

### Resources

[https://www.geeksforgeeks.org/python-classes-and-objects/]